In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vinshee-smritit-text/vinshee_srimit_text-2.csv


In [2]:
#LEARN HOW TO RELOAD THE MODEL 

In [3]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [4]:
from huggingface_hub import notebook_login

notebook_login()
# key_removed_for_privacy

link https://towardsdatascience.com/fine-tune-your-own-llama-2-model-in-a-colab-notebook-df9823a04a32

In [5]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [6]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
model_name = "Abhishekdhaka/llama-2-7b-finetuned"
dataset_name = "/kaggle/input/abhay-structured-data/abhay_structured_data.csv"
# Fine-tuned model name
new_model = "llama-2-7b-finetuned"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"

In [ ]:
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"




In [ ]:
num_train_epochs = 3

In [ ]:

fp16 = False
bf16 = False

In [ ]:
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True


max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"

In [ ]:
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 25
max_seq_length = None


packing = False

In [ ]:
#device_map = {"": 0}
device_map = {"": 0, "optimizer": 1}

In [8]:
import pandas as pd
from datasets import Dataset
df=pd.read_csv('/kaggle/input/ayushi-vinshee-text-data/ayushi_vinshee_text.csv')

dataset = Dataset.from_pandas(df[['text']])

In [9]:
df['text'][0]
print(len(dataset))

189


In [10]:
df.iloc[0]['text']

'[INST] If you are a healthcare professional or medical expert, Provide a response emphasizing the importance of consulting a healthcare professional for post-MRI symptoms. [/INST] I just had an MRI on thursday, Ive had many MRIs in the last 10 years but this was my first experience in a 3.0 Tesla imaging unit. I have relapsing remitting multiple sclerosis and am on Tysabri as my disease modifying drug, I have tested positive for the JC virus {papovavirus} and so my MRIs are to watch out for possible signs of PML.  My head has been pounding off and on since thursday and I was wondering if this is common in higher Tesla MRIs. [INST]  It is possible to get headaches after 3.0 Tesla MRI and several of my patients have also complained about that earlier to me. I am not sure of the exact reason, but may be related to noise or higher magnetic field. It settles down in a few days. Wishing you a favorable MRI report. Please get back if you require any additional information [/INST]'

In [11]:
dataset

Dataset({
    features: ['text'],
    num_rows: 189
})

In [12]:
#SETTINGS


compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

In [13]:
#DOWNLOADING THE MODEL

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:

# LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
# training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard",
    #output_dir="MEDICAL-LLaMA",
    #push_to_hub=True
)


In [ ]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:

trainer.train()

trainer.model.save_pretrained(new_model)

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
25,2.206600
50,2.088000
75,1.994900
100,1.950200
125,1.892100


RESET THE KERNAL

# saving model in hub 

In [1]:
pip install huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [2]:
from huggingface_hub import notebook_login

notebook_login()
# key_removed_for_privacy

In [3]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model.push_to_hub('Abhishekdhaka/llama2vinsheeandayushi')

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

In [ ]:
tokenizer.push_to_hub('Abhishekdhaka/llama2vinsheeandayushi')

In [13]:
name_model='Abhishekdhaka/llama2vinsheeandayushi'
model.push_to_hub(name_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

FileNotFoundError: [Errno 2] No such file or directory: 'llama2vinsheeandayushi'

## sementic search

In [ ]:
!pip install cohere umap-learn altair annoy datasets tqdm

In [ ]:

import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [ ]:
dataset = load_dataset("trec", split="train")
df = pd.DataFrame(dataset)[:1000]
df.head(10)

In [ ]:

api_key = ''

co = cohere.Client(api_key)
embeds = co.embed(texts=list(df['text']),
                  model='embed-english-v2.0').embeddings

In [ ]:
search_index = AnnoyIndex(np.array(embeds).shape[1], 'angular')

for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])
search_index.build(10) # 10 trees
search_index.save('test.ann')

In [ ]:
query = f"Question:'{df.iloc[example_id]['text']}'\nNearest neighbors:"
query_embed = co.embed(texts=[query],
                  model="embed-english-v2.0").embeddings

similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
                                                include_distances=True)
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'], 
                             'distance': similar_item_ids[1]})


print(f"Query:'{query}'\nNearest neighbors:")
results

## Summarization ************

In [ ]:
$ pip install cohere

In [ ]:
import cohere
co = cohere.Client(api_key)

In [ ]:
text ="""It's an exciting day for the development community. Cohere's state-of-the-art language AI is now available through Amazon SageMaker. This makes it easier for developers to deploy Cohere's pre-trained generation language model to Amazon SageMaker, an end-to-end machine learning (ML) service. Developers, data scientists, and business analysts use Amazon SageMaker to build, train, and deploy ML models quickly and easily using its fully managed infrastructure, tools, and workflows.
At Cohere, the focus is on language. The company's mission is to enable developers and businesses to add language AI to their technology stack and build game-changing applications with it. Cohere helps developers and businesses automate a wide range of tasks, such as copywriting, named entity recognition, paraphrasing, text summarization, and classification. The company builds and continually improves its general-purpose large language models (LLMs), making them accessible via a simple-to-use platform. Companies can use the models out of the box or tailor them to their particular needs using their own custom data.
Developers using SageMaker will have access to Cohere's Medium generation language model. The Medium generation model excels at tasks that require fast responses, such as question answering, copywriting, or paraphrasing. The Medium model is deployed in containers that enable low-latency inference on a diverse set of hardware accelerators available on AWS, providing different cost and performance advantages for SageMaker customers.
"""

In [ ]:
response = co.summarize(
    text=text,
    model='command',
    length='medium',
    extractiveness='medium'
)

summary = response.summary

In [ ]:
summary

## CoRank 

In [ ]:
$ pip install cohere

In [ ]:
import cohere
co = cohere.Client('key')


we will get the docs for nns

In [ ]:
query = "What is the capital of the United States?"
docs = [
    "Carson City is the capital city of the American state of Nevada. At the 2010 United States Census, Carson City had a population of 55,274.",
    "The Commonwealth of the Northern Mariana Islands is a group of islands in the Pacific Ocean that are a political division controlled by the United States. Its capital is Saipan.",
    "Charlotte Amalie is the capital and largest city of the United States Virgin Islands. It has about 20,000 people. The city is on the island of Saint Thomas.",
    "Washington, D.C. (also known as simply Washington or D.C., and officially as the District of Columbia) is the capital of the United States. It is a federal district. The President of the USA and many major national government offices are in the territory. This makes it the political center of the United States of America.",
    "Capital punishment (the death penalty) has existed in the United States since before the United States was a country. As of 2017, capital punishment is legal in 30 of the 50 states. The federal government (including the United States military) also uses capital punishment."]

In [ ]:
results = co.rerank(query=query, documents=docs, top_n=3, model='rerank-english-v2.0') # Change top_n to change the number of results returned. If top_n is not passed, all results will be returned.
for idx, r in enumerate(results):
  print(f"Document Rank: {idx + 1}, Document Index: {r.index}")
  print(f"Document: {r.document['text']}")
  print(f"Relevance Score: {r.relevance_score:.2f}")
  print("\n")

## gradio

In [1]:
!pip install text_generation

In [2]:
pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.1/20.1 MB 47.5 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.2/298.2 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.7 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.1-py3-none-any.whl size=5596 sha256=7131ed00480352b4c60b96b6ed9a011f0d439a78310d5d8ee9f822982937fa06
  Stored in directory: /root/.cache/pip/wheels/01/a6/d1/1c0828c304a4283b2c1639a09ad86f83d7c487ef34c6b4a1bf
Successfully built ffmpy
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import io
import IPython.display
from PIL import Image
import base64 
import requests 
requests.adapters.DEFAULT_TIMEOUT = 60

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
hf_api_key = 'key'

In [1]:
import gradio as gr

def greet(name):
    return "Hello " + name + "!"

demo = gr.Interface(
    fn=greet,
    inputs=gr.Textbox(lines=2, placeholder="Name Here..."),
    outputs="text",
)
demo.launch()

ModuleNotFoundError: No module named 'gradio'

the library provide an option for image uplaod also

history: a list of list representing the conversations up until that point. Each inner list consists of two str representing a pair: [user input, bot response].


In [6]:
import random
import gradio as gr

def random_response(message, history):
    return random.choice(["Yes", "No"])

demo = gr.ChatInterface(random_response)

demo.launch(share='True')

Running on local URL:  http://127.0.0.1:7862
Running on public URL: https://3ed3f5709a2c3ca545.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


this is a bit more complex structure shows that you move around the things 

In [ ]:
# import numpy as np
# import gradio as gr


# def flip_text(x):
#     return x[::-1]


# def flip_image(x):
#     return np.fliplr(x)


# with gr.Blocks() as demo:
#     gr.Markdown("Flip text or image files using this demo.")
#     with gr.Tab("Flip Text"):
#         text_input = gr.Textbox()
#         text_output = gr.Textbox()
#         text_button = gr.Button("Flip")
#     with gr.Tab("Flip Image"):
#         with gr.Row():
#             image_input = gr.Image()
#             image_output = gr.Image()
#         image_button = gr.Button("Flip")

#     with gr.Accordion("Open for More!"):
#         gr.Markdown("Look at me...")

#     text_button.click(flip_text, inputs=text_input, outputs=text_output)
#     image_button.click(flip_image, inputs=image_input, outputs=image_output)

# demo.launch()

This is the best user interface which got from course

In [7]:
def format_chat_prompt(message, chat_history, instruction):
    prompt = f"System:{instruction}"
    for turn in chat_history:
        user_message, bot_message = turn
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    return prompt

def respond(message, chat_history, instruction, temperature=0.7):
    prompt = format_chat_prompt(message, chat_history, instruction)
    chat_history = chat_history + [[message, ""]]
    stream = client.generate_stream(prompt,
                                      max_new_tokens=1024,
                                      stop_sequences=["\nUser:", "<|endoftext|>"],
                                      temperature=temperature)
                                      
    acc_text = ""

    for idx, response in enumerate(stream):
            text_token = response.token.text

            if response.details:
                return

            if idx == 0 and text_token.startswith(" "):
                text_token = text_token[1:]

            acc_text += text_token
            last_turn = list(chat_history.pop(-1))
            last_turn[-1] += acc_text
            chat_history = chat_history + [last_turn]
            yield "", chat_history
            acc_text = ""

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(height=240) #just to fit the notebook
    msg = gr.Textbox(label="Prompt")
    with gr.Accordion(label="Advanced options",open=False):
        system = gr.Textbox(label="System message", lines=2, value="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.")
        temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1)
    btn = gr.Button("Submit")
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    btn.click(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot])
    msg.submit(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
#demo.queue().launch(share=True, server_port=int(os.environ['PORT4']))
demo.launch()

Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860
Kaggle notebooks require sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Running on public URL: https://0dc9099d783b5a4b69.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
gr.close_all()

# database

In [ ]:
import sqlite3
#when you do this the database is automatically created in output 
conn = sqlite3.connect('your_database_new.db')


In [ ]:
cursor = conn.cursor()

# Create a users table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS users (
        id INTEGER PRIMARY KEY,
        username TEXT UNIQUE NOT NULL,
        password_hash TEXT NOT NULL
    )
''')

# Commit the changes
conn.commit()


In [ ]:
# Insert data into the users table
cursor.execute("INSERT INTO users (username, password_hash) VALUES (?, ?)", ('example_user', 'hashed_password'))


In [ ]:
conn.close()

In [ ]:

import shutil
database_source = 'your_database.db'
destination_directory = '/kaggle/working/'

shutil.copy(database_source, destination_directory)


In [ ]:
res = cursor.execute("SELECT * FROM users ")
res.fetchall()

## log in 

In [ ]:
!pip install Flask


In [ ]:
!pip install Flask-Login


In [ ]:
from flask import Flask, render_template, request, redirect, url_for, session
app = Flask(__name__)


In [ ]:
import os
app.secret_key = os.urandom(24)


In [ ]:
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user

login_manager = LoginManager()
login_manager.init_app(app)


In [ ]:
class User(UserMixin):
    def __init__(self, id):
        self.id = id
users = {'user1': {'password': 'password1'}}
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        if username in users and users[username]['password'] == password:
            user = User(username)
            login_user(user)
            return redirect(url_for('dashboard'))
        else:
            return 'Invalid login credentials. Please try again.'
    return render_template('login.html')


In [ ]:
@app.route('/dashboard')
@login_required
def dashboard():
    return 'Welcome to the dashboard, {}'.format(current_user.id)


In [ ]:
@app.route('/logout')
@login_required
def logout():
    logout_user()
    return 'You have been logged out. Goodbye!'


In [ ]:
#flask is not working 
app.run()

## log in from gradio 

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import gradio as gr

# Mock user data (replace with your actual user data from your SQLite database)
users = {'user1': {'password': 'password1'}}

# Define a User model for login
class User:
    def __init__(self, username):
        self.username = username

    def authenticate(self, password):
        return self.username in users and users[self.username]['password'] == password

# Your existing functions for data analysis (get_count_ride_type, get_most_popular_stations)
# ...

# Gradio setup
with gr.Blocks() as demo:
    with gr.Row():
        bike_type = gr.Plot()
        station = gr.Plot()

    demo.load(get_count_ride_type, inputs=None, outputs=bike_type)
    demo.load(get_most_popular_stations, inputs=None, outputs=station)

# Define a custom interface for user login
def login_interface():
    username = gr.Textbox(label="Username")
    password = gr.Textbox(label="Password", type="password")
    submit = gr.Button(label="Log In")

    return gr.Interface(
        fn=lambda username, password: User(username).authenticate(password),
        inputs=[username, password],
        outputs=gr.Label(),
        live=True,
        examples=[["user1", "password1"]],
        layout="vertical",
    )

# Run the Gradio application
login_app = login_interface()
login_app.launch()


In [ ]:
pip install psycopg2

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

DB_USER = os.getenv("DB_USER")
DB_USER='/kaggle/working/your_database.db'

connection_string = f"postgresql://{DB_USER}"

In [ ]:


def get_count_ride_type():
    df = pd.read_sql(
    """
        SELECT *
        FROM users

    """,
    con=connection_string
    )
    fig_m, ax = plt.subplots()
    ax.bar(x=df['rideable_type'], height=df['n'])
    ax.set_title("Number of rides by bycycle type")
    ax.set_ylabel("Number of Rides")
    ax.set_xlabel("Bicycle Type")
    return fig_m


def get_most_popular_stations():

    df = pd.read_sql(
        """
    SELECT COUNT(ride_id) as n, MAX(start_station_name) as station
    FROM RIDES
    WHERE start_station_name is NOT NULL
    GROUP BY start_station_id
    ORDER BY n DESC
    LIMIT 5
    """,
    con=connection_string
    )
    fig_m, ax = plt.subplots()
    ax.bar(x=df['station'], height=df['n'])
    ax.set_title("Most popular stations")
    ax.set_ylabel("Number of Rides")
    ax.set_xlabel("Station Name")
    ax.set_xticklabels(
        df['station'], rotation=45, ha="right", rotation_mode="anchor"
    )
    ax.tick_params(axis="x", labelsize=8)
    fig_m.tight_layout()
    return fig_m

In [ ]:
def method_here():
    print('hi you are welcome')

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    #with gr.Row():

    demo.load(method_here, inputs=None, outputs=None)
    #demo.load(get_count_ride_type, inputs=None, outputs=None)
    

demo.launch()

In [ ]:
#THIS IS BASIC LOG IN PAGE
import gradio as gr

# Mock user data (replace with your actual user data from your database)
users = {'user1': {'password': 'password1'}, 'user2': {'password': 'password2'}}

# Function to check user credentials
def authenticate(username, password):
    if username in users and users[username]['password'] == password:
        return f"Hi, {username}!"
    else:
        return "Invalid credentials. Please try again."

# Gradio interface
interface = gr.Interface(
    fn=authenticate,
    inputs=[
        gr.Textbox(label="Username"),
        gr.Textbox(label="Password", type="password")
    ],
    outputs=gr.Textbox(default=""),
    live=True,
    title="User Login",
    description="Enter your username and password.",
)

# Launch the Gradio interface
interface.launch()


#Experimental : for user login 

In [ ]:
import gradio as gr


users = {'user1': {'password': 'password1'}, 'user2': {'password': 'password2'}}


def chatbot(input_text):

    response = "Chatbot: You said, '{}'. Please replace this with your chatbot's response.".format(input_text)
    return response


def authenticate(username, password):
    if username in users and users[username]['password'] == password:
        return True
    else:
        return False


chatbot_interface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="You:"),
    outputs=gr.Textbox(label="Chatbot:"),
    live=True,
    title="Chatbot",
    description="Enter your messages to chat with the bot.",
)


login_interface = gr.Interface(
    fn=authenticate,
    inputs=[
        gr.Textbox(label="Username"),
        gr.Textbox(label="Password", type="password")
    ],
    outputs=gr.Textbox(default="enter your password"),
    live=True,
    title="User Login",
    description="Enter your username and password.",
)


login_interface.launch()


authenticated = False


while not authenticated:
    if login_interface.get_latest_output() == "True":
        authenticated = True

chatbot_interface.launch()


In [ ]:

import gradio as gr


users = {'user1': {'password': 'password1'}, 'user2': {'password': 'password2'}}

def chatbot(input_text):

    response = "Chatbot: You said, '{}'. Please replace this with your chatbot's response.".format(input_text)
    
    return response


def authenticate(username, password):
    if username in users and users[username]['password'] == password:
        chatbot_interface.launch()
        demo.close()
        chatbot_interface.close()
        return "welcome"
    else:
        return "wrong credentials"



chatbot_interface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="You:"),
    outputs=gr.Textbox(label="Chatbot:"),
    live=True,
    title="Chatbot",
    description="Enter your messages to chat with the bot.",
)
with gr.Blocks() as demo:
    #chatbot = gr.Chatbot(height=240) #just to fit the notebook
    user_name = gr.Textbox(label="User Name")
    user_password = gr.Textbox(label="User Password")
#     with gr.Accordion(label="Advanced options",open=False):
#         system = gr.Textbox(label="System message", lines=2, value="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.")
#         temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1)
    btn = gr.Button("Login")
    clear = gr.ClearButton(components=[user_name, user_password], value="Reset")
    correct_id = gr.Textbox(label="Correct ID")
    btn.click(authenticate, inputs=[user_name, user_password], outputs=[correct_id])
    print(correct_id)
    #msg.submit(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
#demo.queue().launch(share=True, server_port=int(os.environ['PORT4']))
demo.launch()

In [ ]:
#my try method 2
import gradio as gr


users = {'user1': {'password': 'password1'}, 'user2': {'password': 'password2'}}


def chatbot(input_text):

    response = "Chatbot: You said, '{}'. Please replace this with your chatbot's response.".format(input_text)
    return response


def authenticate(username, password):
    if username in users and users[username]['password'] == password:
        chatbot_interface.launch()
        return "welcome"
    else:
        return "wrong credentials"


    
login_interface = gr.Interface(
    fn=authenticate,
    inputs=gr.Textbox(label="username:"),
    outputs=gr.Textbox(label="password:"),
    live=True,
    title="Chatbot",
    description="Enter your messages to chat with the bot.",
)

chatbot_interface = gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(label="You:"),
    outputs=gr.Textbox(label="Chatbot:"),
    live=True,
    title="Chatbot",
    description="Enter your messages to chat with the bot.",
)
with gr.Blocks() as demo:
    #chatbot = gr.Chatbot(height=240) #just to fit the notebook
    user_name = gr.Textbox(label="User Name")
    user_password = gr.Textbox(label="User Password")
#     with gr.Accordion(label="Advanced options",open=False):
#         system = gr.Textbox(label="System message", lines=2, value="A conversation between a user and an LLM-based AI assistant. The assistant gives helpful and honest answers.")
#         temperature = gr.Slider(label="temperature", minimum=0.1, maximum=1, value=0.7, step=0.1)
    btn = gr.Button("Login")
    clear = gr.ClearButton(components=[user_name, user_password], value="Reset")
    correct_id = gr.Textbox(label="Correct ID")
    btn.click(authenticate, inputs=[user_name, user_password], outputs=[correct_id])
    print(correct_id)
    #msg.submit(respond, inputs=[msg, chatbot, system], outputs=[msg, chatbot]) #Press enter to submit
gr.close_all()
#demo.queue().launch(share=True, server_port=int(os.environ['PORT4']))
demo.launch()

In [ ]:
import gradio as gr


def process_inputs(input1, input2, input3):

    result = f"Input 1: {input1}\nInput 2: {input2}\nInput 3: {input3}"
    
    
    iface2.launch()
    return result

def process_inputss(input1, input2, input3):
    iface.close()

    result = f"Input 11111: {input1}\nInput 2: {input2}\nInput 3: {input3}"

    return result


input1 = gr.inputs.Textbox(label="Input 1")
input2 = gr.inputs.Textbox(label="Input 2")
input3 = gr.inputs.Textbox(label="Input 3")


iface = gr.Interface(
    fn=process_inputs,
    inputs=[input1, input2, input3],
    outputs="text"
)
iface2 = gr.Interface(
    fn=process_inputss,
    inputs=[input1, input2, input3],
    outputs="text"
)

# Launch the Gradio interface
iface.launch()


In [ ]:
import gradio as gr
import threading


valid_username = "user123"
valid_password = "password123"


def check_credentials(username, password):
    return username == valid_username and password == valid_password
username = gr.inputs.Textbox(label="Input 1")
password = gr.inputs.Textbox(label="Input 2")


def login_interface(username, password):
    if check_credentials(username, password):

        chat_thread = threading.Thread(target=chat_interface.launch())
        chat_thread.start()


        login_interface.close()

        return "Login successful. Redirecting to the chat interface..."
    else:
        return "Invalid username or password. Please try again."


def chat_interface(input_text):
    # Implement your chatbot logic here
    response = "Chatbot response: You said - " + input_text
    return response


login_interface = gr.Interface(fn=login_interface, inputs=[username, password], outputs=None, live=False)
chatbot_interface = gr.Interface(fn=chat_interface, inputs="text", outputs="text", live=True)

if __name__ == "__main__":

    login_thread = threading.Thread(target=login_interface.launch)
    login_thread.start()


In [ ]:
#if you want to upload the training report NOTE: it will not upload the saved model you have to save the model locally
trainer.push_to_hub()

In [ ]:
#since we are using 4bit so this code will not work 
#model.push_to_hub('Abhishekdhaka/MEDICAL-LLaMA')

In [ ]:
prompt = "मुझे इस दर्द के लिए कौन सी दवाई सबसे अच्छी है?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
import os
output_directory = '/kaggle/working/model_saving'
os.makedirs(output_directory, exist_ok=True)


In [ ]:
from huggingface_hub import HfApi
api = HfApi()
from huggingface_hub import create_repo,delete_repo
#delete_repo(repo_id="Abhishekdhaka/MedicalLLaMA_data")
create_repo("Abhishekdhaka/MedicalLLaMA_data", repo_type="space",space_sdk='docker')

#To delete a resposity


In [ ]:


api.upload_folder(
    folder_path="/kaggle/working/results",
    repo_id="Abhishekdhaka/MedicalLLaMA_data",
    repo_type="space",
)

In [ ]:
# to download the complete repo back you can use this code ##IT IS NOT WORKING AS OF NOW
from huggingface_hub import snapshot_download
snapshot_download(repo_id="Abhishekdhaka/MedicalLLaMA_data", repo_type="space")

snapshot_download(repo_id="google/fleurs")